In [ ]:
from collections import namedtuple
from websocket import create_connection
from typing import List
from pyspark.sql.types import *
from pyspark.sql.window import Window
import datetime
import os
import json
import requests
import re
import pandas as pd
import pyspark.sql.functions as f
import time

In [ ]:
os.environ["BASE_URL"] = "https://valsys.iai.cppib.io/"
def authenticate(username, password):
    # make the request
    auth_url =  os.environ["BASE_URL"] + "users/login"
    headers = {
      'username': username,
      'password': password
    }

    # decode into an object and validate
    response = requests.request("GET", auth_url, headers=headers, data=None)
    auth_response = json.loads(response.text.encode('utf8'), object_hook=lambda d: namedtuple('X', d.keys())(*d.values()))
    if auth_response.status != "success":
        print("ERROR:", auth_response.message)#
        return
    
    # set access token as environment variable
    os.environ["TOKEN"] = auth_response.data.AccessToken

In [ ]:
# machine_model_df = spark.read.table("model_engine.valsys_machine_models_v2").selectExpr("keydriver_source as source", "model_id").distinct()
machine_model_df = spark.read.table("model_engine.valsys_machine_models_om").selectExpr("keydriver_source as source", "model_id").distinct()


final_om_df = spark.read.table("model_engine.operating_model_config")

w_func = Window.partitionBy("source").orderBy(f.col("last_updated").desc())
latest_timestamp_df = final_om_df.select("source", "last_updated").distinct()
latest_timestamp_df = latest_timestamp_df.withColumn("update_rank", f.rank().over(w_func)).filter("update_rank = 1").drop("update_rank")

# module_df = module_df.withColumn("model_id", f.when(f.col("ticker") == "ANGI US", f.lit("0x21fa898")).otherwise(f.col("model_id")))
final_om_df = final_om_df.join(latest_timestamp_df, ["source", "last_updated"])
module_df = final_om_df.join(machine_model_df, ["source"]).withColumn("period_year", f.col("period_year").cast(IntegerType())).filter("ticker = 'W US'")

# check for new source
# for tbl in list(spark.catalog.listTables("model_engine")):
#   if "valsys_modules" in tbl.name:
#     df2 = spark.read.table("model_engine.valsys_machine_models").select('ticker', 'keydriver_source', 'templateID').drop_duplicates()
#     df = df.join(df2, [df2.keydriver_source == df.source], how="leftanti")

In [ ]:
username = "overlord@cppib.com"
password = "DDIOverlord!23"

authenticate(username, password)

In [ ]:
parent_module_name = "Key Drivers (Input)"
module_name = "Operating Model"
model_info = [{"source": x.source, "model_id": x.model_id, "ticker": x.ticker} for x in module_df.select("source", "model_id", "ticker").distinct().collect()]

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2471821865463924> in <module> 
 1 parent_module_name = "Key Drivers (Input)" 
 2 module_name = "Operating Model" 
 ----> 3 model_info = [ { "source" : x . source , "model_id" : x . model_id , "ticker" : x . ticker } for x in module_df . select ( "source" , "model_id" , "ticker" ) . distinct ( ) . collect ( ) ] 

 NameError : name 'module_df' is not defined

In [ ]:
for mi in model_info:
  model_id = mi["model_id"]
#   model_id = "0x21fa898"
  print("processing model id: {}, ticker: {}".format(model_id, mi["ticker"]))
  case_id = pull_model_information(model_id)
  case = pull_case(case_id)
  
  is_module = case.pull_module(parent_module_name)
  om_module = case.pull_module(module_name)
  module_id = is_module.uid
  
  if om_module is not None:
    print("removing {} module".format(module_name))
    om_module_id = om_module.uid
    remove_module(model_id, case_id, om_module_id, module_id, module_name)
  
  time.sleep(1)

In [ ]:
module_pdf = module_df.toPandas().sort_values(by=["source", "item_order", "period_year"])
module_name = "Operating Model"
key_metrics = ["Revenue Growth, %", "Gross Margin, %", "SG&A / sales", "R&D / sales", "Capex / sales"]
key_metrics_format = {"fontWeight":"bold","fontStyle":"normal","textAlign":"right","textDecoration":"none","valFormat":"Percentage","unit":"Raw","decimalPlaces":1}
om_module_info = []

for m_info in model_info:
  #   model_id = "0x21fa898"
  model_id = m_info["model_id"]
  kd_source = m_info["source"]
  om_dict = dict()
  
  print("processing: {}".format(kd_source))
   
  model_pdf = module_pdf.loc[module_pdf["model_id"] == model_id]
  # Pull the first case uid
  case_id = pull_model_information(model_id)
  # Pull the case data, the case returned holds the model data packaged as a Case object
  case = pull_case(case_id)
  # Select the income statement module as we want to add a new module as a revenue driver
  root_module = case.pull_module(parent_module_name)
  
  #create OM Module
  om_module = root_module.add_child_module(module_name, model_id, case_id)
  
  item_pdf = model_pdf[["kpi_name", "item_order"]].drop_duplicates().sort_values(by=["item_order"])
  
  om_item_lst = []

  for i,v in item_pdf.iterrows():
    om_item_dict = dict()
    id_dict = dict()

    item_name = v["kpi_name"]
    item_order = v["item_order"]

    om_item_dict["kpi_name"] = item_name
    item_obj = om_module.add_item(item_name, item_order, model_id, case_id)

    if item_name in key_metrics:

      for idx, cell in enumerate(item_obj.facts):
        cell.fmt = json.dumps(key_metrics_format)
        item_obj.facts[idx] = cell

      print("Editing Format: {}".format(item_name))
      item_obj.edit_format(model_id, case_id)

    om_item_dict["item_obj"] = item_obj
    om_item_lst.append(om_item_dict)

  id_lst = []
  for omi in om_item_lst:
    kpi_name = omi["kpi_name"]
    item_obj = omi["item_obj"]

    tmp_pdf = model_pdf.loc[model_pdf["kpi_name"] == kpi_name].sort_values(by=["period_year"])

    if len(tmp_pdf.index) > 0:
      id_dict_lst = []
      for idx, cell in enumerate(item_obj.facts):

        for i,v in tmp_pdf.iterrows():
          if cell.period == int(v["period_year"]):
            cell.formula = v["formula"]
            print(cell.formula)

            if v["period_name"] != "reported" and kpi_name in key_metrics:
              id_info = dict()
              id_info["identifier"] = cell.identifier
              id_info["period_year"] = cell.period
              id_dict_lst.append(id_info)
        item_obj.facts[idx] = cell
      item_obj.apply_edits(model_id, case_id)

      if len(id_dict_lst) > 0:
        id_dict = dict()
        id_dict[kpi_name] = id_dict_lst
        id_lst.append(id_dict)

    if kpi_name in key_metrics:
      om_dict["source"] = kd_source
      om_dict["model_id"] = model_id
      om_dict["case_id"] = case_id
      om_dict["case"] = case
      om_dict["metric"] = kpi_name

      for i in id_lst:
        if kpi_name in i.keys():
          om_dict["id_info"] = i[kpi_name]

      om_module_info.append(om_dict)

  time.sleep(1)

In [ ]:
keydriver_pdf = (spark.read.table("model_engine.xl_config_raw_om")
                     .filter((f.col("item_description").isin(*key_metrics)) & (f.col("source_type") == 'Operating Model') & (f.col("forecast") == 1))
                     .select("source", "item_description", "hybrid_formula_clean", "case")
                     .orderBy("source", "item_description")
                     .distinct()
                     .toPandas())

kd_info = dict()
for idx, val in keydriver_pdf.iterrows():
  src = val["source"]
  metric = val["item_description"]
  case = val["case"]
  fmla = val["hybrid_formula_clean"]
  
  fmla_info = dict()
  fmla_info[case] = fmla
  
  if src not in kd_info.keys():
    kd_info[src] = dict()
  
  if src in kd_info.keys() and metric not in kd_info[src].keys():
    kd_info[src][metric] = dict()
  
  kd_info[src][metric].update(fmla_info)

In [ ]:
for omi in om_module_info:
  src = omi["source"]
  metric = omi["metric"]
  id_lst = omi["id_info"]
  case = omi["case"]
  keydriver_module = case.pull_module(parent_module_name)
  
  for li in ["Base", "Upside", "Downside"]:
    item_name = "{} ({})".format(metric, li)
    line_item = keydriver_module.pull_item_by_name(item_name)
    scenario = li.lower()
    if src in kd_info.keys() and metric in kd_info[src].keys():
      print("processing source: {}, metric: {}, case: {}".format(src, metric, scenario))

      fmla = kd_info[src][metric][scenario]
      for idx, cell in enumerate(line_item.facts):
        for il in id_lst:
          if cell.period == il["period_year"]:
            cell_fmla = il["identifier"]
            if fmla is not None and len(fmla) > 0:
              cell_fmla = "{} {}".format(il["identifier"], fmla)
            cell.formula = cell_fmla
            line_item.facts[idx] = cell
      line_item.apply_edits(omi["model_id"], omi["case_id"])
  recalculate_model(model_id)